# Create & Use Log Functions

This is a silly, lil' example that I am currently developing to be able to interact with the current logging tools.  I'm also using it to determine and communicate possible design decisions.

__TODO__: Finish a rough draft of this for review by the others.

For this method, we impose the following criteria for determining what information should be written with which debug log levels
* `LOG_LEVEL_MIN_DEBUG` - Insights for tracing method's progress.  Potentially useful for careful users and method developers.  This should not be an overwhelming amount of information.
* `LOG_LEVEL_MIN_DEBUG` + 1 - Reporting coordinates of a parameter-space point.  For high-dimensional spaces, such information could be substantially long and could render the log ineffective for general use.  Users might find this useful for checking setups.
* `LOG_LEVEL_MIN_DEBUG` + 2 - Reporting internal and intermediate data for helping developers develop and debug the method.

## Define low-level method implementation
Note that at the level of this low-level layer there's not much utility in logging the starting point, for example, since this layer of software does not know the mapping of parameter names onto coordinates.  However, it makes sense to log optional configuration values in this layer since this code knows how to manage the values of these.

In [1]:
import numpy as np

import poptus

def my_method(model, x0, numerics, expert, logger, **kwargs):
    # ----- HARDCODED VALUES
    LOG_TAG = "MyMethod"

    DEBUG_0 = 0
    DEBUG_LONG = 1
    DEBUG_WEEDS = 2

    NOISE_THRESHOLD = 1.0e-2

    # ----- CREATE LOG FUNCTIONS
    log, log_debug, warn, log_and_abort = \
        poptus.create_log_functions(logger, LOG_TAG)

    # ----- HANDLE OPTIONAL ARGS & ERROR CHECK CONFIG
    delta_0 = numerics["initial_delta"]
    max_iters = numerics["max_iters"]
    gtolr = expert["gtolr"]
    if delta_0 <= 0.0:
        log_and_abort(ValueError, "Initial delta must be positive")
    elif max_iters <= 0:
        log_and_abort(ValueError, "Number of iterations must be positive")
    if gtolr > 1.0e-3:
        warn("My what a large gradient tolerance you've provided")

    # ----- CREATE LOG FUNCTIONS & LOG CONFIGURATION
    log("Numerics")
    log("-" * 60)
    for key, value in numerics.items():
        log(f"{key:<15}{value}")
    log("")
    log("Expert Settings")
    log("-" * 60)
    for key, value in expert.items():
        log(f"{key:<15}{value}")

    # ----- START EXECUTING METHOD
    log("")
    log_debug("Let's get started!", DEBUG_0)
    x = np.array(x0)
    for i in range(max_iters):
        x *= 0.5
        log(f"Iteration {i+1} = {x}")
        log_debug("We divided by two to get this", DEBUG_WEEDS)
        solution = model(1.1, x)
        log_debug(f"Iteration {i+1} solution = {solution}", DEBUG_0)
        if np.fabs(solution) < 1.0e-1:
            warn("Solution is suspiciously small")

    log(f"solution = {solution} at x = {x}")
    if np.fabs(solution) <= NOISE_THRESHOLD:
        log_and_abort(RuntimeError, "Solution too small")

    return solution, x

## Logging to standard output and error

When using the low-level interface, users are made aware of the `poptus` logging facilities directly and are required to instantiate a logger.  

In [2]:
def model(nu, x):
    m, b = x
    return m * nu + b

# Ensure that ordering is in accord with how model() uses x
x0 = [-2.2, 1.1]
numerics = {
    "initial_delta": 1.0e-2,
    "max_iters": 4
}
expert = {
    "gtolr": 1.0e-2
}
config_std = {
    "Level": poptus.LOG_LEVEL_MIN_DEBUG + 1
}

logger = poptus.create_logger(config_std)
_ = my_method(model, x0, numerics, expert, logger)

[MyMethod] WARNING - My what a large gradient tolerance you've provided
[MyMethod] Numerics
[MyMethod] ------------------------------------------------------------
[MyMethod] initial_delta  0.01
[MyMethod] max_iters      4
[MyMethod] 
[MyMethod] Expert Settings
[MyMethod] ------------------------------------------------------------
[MyMethod] gtolr          0.01
[MyMethod] 
[MyMethod] Let's get started!
[MyMethod] Iteration 1 = [-1.1   0.55]
[MyMethod] Iteration 1 solution = -0.6600000000000001
[MyMethod] Iteration 2 = [-0.55   0.275]
[MyMethod] Iteration 2 solution = -0.33000000000000007
[MyMethod] Iteration 3 = [-0.275   0.1375]
[MyMethod] Iteration 3 solution = -0.16500000000000004
[MyMethod] Iteration 4 = [-0.1375   0.06875]
[MyMethod] Iteration 4 solution = -0.08250000000000002
[MyMethod] WARNING - Solution is suspiciously small
[MyMethod] solution = -0.08250000000000002 at x = [-0.1375   0.06875]


## Define high-level method interface

When using the low-level interface, users are made aware of the `poptus` logging facilities directly and are required to instantiate a logger.  A method can also provide a high-level interface that allows users to interact with methods through simple data structures rather than code.  One benefit of this is that users are only exposed to loggers through the requirement that the simply specify their logging requirements in their data structures.

Note that it makes more sense to log in the high-level layer at least some of the configuration information, such as the starting point, since that layer knows the mapping between parameter names and coordinates.

In [3]:
import pandas as pd

def run_my_method(filename_json, parameter_space, model):
    # ----- HARDCODED VALUES
    LOG_TAG = "MyMethod"

    DEBUG_0 = 0
    DEBUG_LONG = 1
    DEBUG_WEEDS = 2

    with open(filename_json, "r") as fptr:
        full_cfg = json.load(fptr)

    method_cfg = full_cfg["MyMethod"]

    numerics = method_cfg["Numerics"]
    expert = method_cfg["Expert"]
    logger = poptus.create_logger(method_cfg["Logging"])
    _, log_debug, _, _ = \
        poptus.create_log_functions(logger, "MyMethod")

    # Convert given starting point from unordered dict that maps parameter names
    # to coordinates to ordered array with no notion of parameter names.
    # for the low-level interface
    x0 = method_cfg["StartingPoint"]
    x0_ordered = [x0[p] for p in parameter_space]

    # ----- LOG CONFIGURATION INFORMATION
    # If our method will be used on models with high-dimensional parameter spaces,
    # then logging the starting point might render the log useless.  Therefore,
    # we only print for higher log levels.
    log_debug("Starting Point", DEBUG_LONG)
    log_debug("-" * 60, DEBUG_LONG)
    for parameter, value in x0.items():
        log_debug(f"{parameter:<30}{value:<30}", DEBUG_LONG)
    log_debug("", DEBUG_LONG)

    solution, x_hat = my_method(model, x0_ordered, numerics, expert, logger)

    # Convert ordered parameter-space point to unordered DataFrame with parameter
    # names mapped onto coordinates.
    x_hat_df = pd.DataFrame(data=x_hat, index=parameter_space,
                            columns=["x_hat"]).T
    
    return solution, x_hat_df

We imagine that for a particular application a user must provide configuration information for both their model and the method that will study the model.  In this particular case, both configurations are provided in a single JSON-format file with contents

In [4]:
# Hide inputs
import json

FILENAME_JSON = "MyMethodTest.json"

# You can alter the contents of this and rerun notebook to see
# the effect the changes have on the output.
TO_JSON = {
    "MyMethod": {
        "StartingPoint": {
            "b":   1.1,
            "m":  -2.2
        },
        "Numerics": {
            "initial_delta": 2.5e-3,
            "max_iters":     5
        },
        "Expert": {
            "gtolr": 1.0e-9
        },
        "Logging": {
            "Level": 3
        }
    },
    "Model": {
        "ParameterSpace":  ["m", "b"],
        "n_basis":            20,
        "budget":            500,
        "stopping_criteria":   1.0e-5,
        "Logging": {
            "Level": 1
        }
    }
}
with open(FILENAME_JSON, "w") as fptr:
    json.dump(TO_JSON, fptr, indent=4)

print()
print(f"Contents of FILENAME_JSON = {FILENAME_JSON}")
print()
with open("MyMethodConfig.json", "r") as fptr:
    for line in fptr.readlines():
        print(line.rstrip("\n"))


Contents of FILENAME_JSON = MyMethodTest.json

{
    "MyMethod": {
        "StartingPoint": {
            "b": 1.1,
            "m": -2.2
        },
        "Numerics": {
            "initial_delta": 0.0025,
            "max_iters": 5
        },
        "Expert": {
            "gtolr": 1e-09
        },
        "Logging": {
            "Level": 3
        }
    },
    "Model": {
        "ParameterSpace": [
            "m",
            "b"
        ],
        "n_basis": 20,
        "budget": 500,
        "stopping_criteria": 1e-05,
        "Logging": {
            "Level": 1
        }
    }
}


An application built for using {{poptus}} methods with a particular model might look like the following.  We assume that `FILENAME_JSON` is provided to the application by the user.

In [5]:
# ----- Load model information
# This would likely be provided by the model developer in Python code that we
# would import into the application.
with open(FILENAME_JSON, "r") as fptr:
    configuration = json.load(fptr)
model_cfg = configuration["Model"]
parameter_space = model_cfg["ParameterSpace"]

model_logger = poptus.create_logger(model_cfg["Logging"])

log, _, _, _ = poptus.create_log_functions(model_logger, "Model")
log("Model Parameter Space")
log("-" * 80)
for i, parameter in enumerate(parameter_space):
    log(f"e_{i+1}\t{parameter}")

def model(nu, x):
    # This should match the parameter space definition that we loaded.
    m, b = x
    return m * nu + b

try:
    solution, x_hat_df = run_my_method(FILENAME_JSON, parameter_space, model)
    display(x_hat_df)
except Exception:
    print()
    print("MyMethod failed to find a solution.  Please check log for more details.")
    print()

[Model] Model Parameter Space
[Model] --------------------------------------------------------------------------------
[Model] e_1	m
[Model] e_2	b
[MyMethod] Starting Point
[MyMethod] ------------------------------------------------------------
[MyMethod] b                             1.1                           
[MyMethod] m                             -2.2                          
[MyMethod] 
[MyMethod] Numerics
[MyMethod] ------------------------------------------------------------
[MyMethod] initial_delta  0.0025
[MyMethod] max_iters      5
[MyMethod] 
[MyMethod] Expert Settings
[MyMethod] ------------------------------------------------------------
[MyMethod] gtolr          1e-09
[MyMethod] 
[MyMethod] Let's get started!
[MyMethod] Iteration 1 = [-1.1   0.55]
[MyMethod] Iteration 1 solution = -0.6600000000000001
[MyMethod] Iteration 2 = [-0.55   0.275]
[MyMethod] Iteration 2 solution = -0.33000000000000007
[MyMethod] Iteration 3 = [-0.275   0.1375]
[MyMethod] Iteration 3 soluti

,m,b
x_hat,-0.06875,0.034375


## Log method information to file

Note that in this case any error messages produced by the method will be written to both the log file and standard error.

In [6]:
# Hide inputs
import json

FILENAME_JSON = "MyMethodTest.json"
FILENAME_LOG = "MyMethodTest.log"

# You can alter the contents of this and rerun notebook to see
# the effect the changes have on the output.
TO_JSON = {
    "MyMethod": {
        "StartingPoint": {
            "b":   1.1,
            "m":  -2.2
        },
        "Numerics": {
            "initial_delta": 2.5e-3,
            "max_iters":     10
        },
        "Expert": {
            "gtolr": 1.0e-9
        },
        "Logging": {
            "Filename": FILENAME_LOG,
            "Overwrite": True,
            "Level": 3
        }
    },
    "Model": {
        "ParameterSpace":  ["m", "b"],
        "n_basis":            20,
        "budget":            500,
        "stopping_criteria":   1.0e-5,
        "Logging": {
            "Level": 1
        }
    }
}
with open(FILENAME_JSON, "w") as fptr:
    json.dump(TO_JSON, fptr, indent=4)

print()
print(f"Contents of FILENAME_JSON = {FILENAME_JSON}")
print()
with open(FILENAME_JSON, "r") as fptr:
    for line in fptr.readlines():
        print(line.rstrip("\n"))


Contents of FILENAME_JSON = MyMethodTest.json

{
    "MyMethod": {
        "StartingPoint": {
            "b": 1.1,
            "m": -2.2
        },
        "Numerics": {
            "initial_delta": 0.0025,
            "max_iters": 10
        },
        "Expert": {
            "gtolr": 1e-09
        },
        "Logging": {
            "Filename": "MyMethodTest.log",
            "Overwrite": true,
            "Level": 3
        }
    },
    "Model": {
        "ParameterSpace": [
            "m",
            "b"
        ],
        "n_basis": 20,
        "budget": 500,
        "stopping_criteria": 1e-05,
        "Logging": {
            "Level": 1
        }
    }
}


In [7]:
import functools

# ---- USER-PROVIDED MODEL FUNCTION
# This would likely be provided by the model developer in Python code that we
# would import into the application.
def original_model(nu, x):
    """
    Imagine that the model is developed to accept points with mapping
    of parameter names to coordinates explicitly provided.
    """
    m = x["m"]
    b = x["b"]
    return m * nu + b

def model_wrapper(nu, x_ordered, parameter_space):
    """
    To interface the model with methods that expect points as ordered
    arrays with no mapping, we need to wrap the original model.
    """
    x = dict(zip(parameter_space, x_ordered))
    return original_model(nu, x)

# ----- Load model information
with open(FILENAME_JSON, "r") as fptr:
    configuration = json.load(fptr)
model_cfg = configuration["Model"]

# The users specify a (likely arbitrary) ordering
parameter_space = model_cfg["ParameterSpace"]
model = functools.partial(model_wrapper, parameter_space=parameter_space)

model_logger = poptus.create_logger(model_cfg["Logging"])

log, _, _, _ = poptus.create_log_functions(model_logger, "Model")
log("Model Parameter Space")
log("-" * 80)
for i, parameter in enumerate(parameter_space):
    log(f"e_{i+1}\t{parameter}")

try:
    solution, x_hat_df = run_my_method(FILENAME_JSON, parameter_space, model)
    display(x_hat_df)
except Exception:
    print()
    print("MyMethod failed to find a solution.  Please check log for more details.")
    print()

[MyMethod] ERROR - Solution too small


[Model] Model Parameter Space
[Model] --------------------------------------------------------------------------------
[Model] e_1	m
[Model] e_2	b
[POptUS] WARNING - Overwriting /Users/jared/Projects/POptUS/POptUS_public/book/notebooks/MyMethodTest.log

MyMethod failed to find a solution.  Please check log for more details.



The contents of the method's log file are as follows.

In [8]:
# Hide inputs

with open(FILENAME_LOG, "r") as fptr:
    for line in fptr.readlines():
        print(line[:-1])

[MyMethod] Starting Point
[MyMethod] ------------------------------------------------------------
[MyMethod] b                             1.1                           
[MyMethod] m                             -2.2                          
[MyMethod] 
[MyMethod] Numerics
[MyMethod] ------------------------------------------------------------
[MyMethod] initial_delta  0.0025
[MyMethod] max_iters      10
[MyMethod] 
[MyMethod] Expert Settings
[MyMethod] ------------------------------------------------------------
[MyMethod] gtolr          1e-09
[MyMethod] 
[MyMethod] Let's get started!
[MyMethod] Iteration 1 = [-1.1   0.55]
[MyMethod] Iteration 1 solution = -0.6600000000000001
[MyMethod] Iteration 2 = [-0.55   0.275]
[MyMethod] Iteration 2 solution = -0.33000000000000007
[MyMethod] Iteration 3 = [-0.275   0.1375]
[MyMethod] Iteration 3 solution = -0.16500000000000004
[MyMethod] Iteration 4 = [-0.1375   0.06875]
[MyMethod] Iteration 4 solution = -0.08250000000000002
[MyMethod] WARNING -